# Credit River Conservation's Data Analysis

The main goal of this notebook is to analyze the data given by Credit River Conservation.

In [2]:
import pandas as pd
import numpy as np
import os

## Precipitation Missing values analyze

In [2]:
folder_path = 'Ontario_Data/Credit_River_Conservation/Precipitation'
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=',')
    
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d/%Y %I:%M:%S %p').copy()
    df = df[(df['Datetime'].dt.month >= 4) & (df['Datetime'].dt.month <= 11)]

    df_gap = pd.DataFrame(columns=['Date', 'Date_Prev', 'Gap', 'Num_Gaps'])
    rows = []

    for i, date in enumerate(df['Datetime'].unique()):
        if(i > 0):
            gap = date - date_prev
            if(gap.seconds/60 > 60):
                rows.append([date, date_prev, gap, int((gap.seconds/60 + gap.days*24*60)/15)+1])
        date_prev = date
    for row in rows:
        df_gap.loc[len(df_gap)] = row
    display(df_gap)
    print(f'Total Gaps: {df_gap.Num_Gaps.sum()}')    

,Date,Date_Prev,Gap,Num_Gaps


Total Gaps: 0


,Date,Date_Prev,Gap,Num_Gaps


Total Gaps: 0


,Date,Date_Prev,Gap,Num_Gaps


...

Total Gaps: 0


,Date,Date_Prev,Gap,Num_Gaps


Total Gaps: 0


,Date,Date_Prev,Gap,Num_Gaps


Total Gaps: 0


## Hydrometric Water Level Missing Values analyze

In [4]:
from datetime import datetime
total_measures = (datetime(2021,12,1) - datetime(2021,4,1))*3*24*4
total_measures

datetime.timedelta(days=70272)

In [6]:
gap_choice = 15

folder_path = 'Ontario_Data/Credit_River_Conservation/Hydrometric'
folder_list = os.listdir(folder_path)
for file in os.listdir(folder_path):
    print(f'FILE: {file}')
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=',', low_memory=False)
    try:
        df = df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis='columns')
    except:
        pass
    df_level = df.iloc[:, :3].copy()  # Use .copy() to avoid SettingWithCopyWarning
    df_level['Datetime'] = pd.to_datetime(df_level['Date'] + ' ' + df_level['Time'], format='%m/%d/%Y %I:%M:%S %p')
    df_level = df_level[pd.to_numeric(df_level['Stage (m)'], errors='coerce').notnull()]
    df_level = df_level[df_level.notna().any(axis=1)]
    df_level = df_level[(df_level['Datetime'].dt.month >= 4) & (df_level['Datetime'].dt.month <= 11)]
    print(f'LEN: {len(df_level.Datetime.unique())}')

    df_gap = pd.DataFrame(columns=['Date', 'Date_Prev', 'Gap', 'Num_Gaps'])
    rows = []
    date_prev = 0
    for i, date in enumerate(df_level['Datetime'].unique()):
        if(i > 0):
            gap = date - date_prev
            if((gap.seconds/60 != gap_choice or gap.days > 0) and gap.days < 121):
                rows.append([date, date_prev, gap, int((gap.seconds/60 + gap.days*24*60)/15)-1])
        date_prev = date
    for row in rows:
        df_gap.loc[len(df_gap)] = row
    display(df_gap)
    print(f'Total Gaps: {df_gap.Num_Gaps.sum()}\n')

FILE: dta_YorkU_June2024(Credit Grange).csv
LEN: 70256


,Date,Date_Prev,Gap,Num_Gaps
0,2023-10-27 18:00:00,2023-10-27 13:45:00,0 days 04:15:00,16


Total Gaps: 16

FILE: dta_YorkU_June2024(Credit King).csv
LEN: 62965


,Date,Date_Prev,Gap,Num_Gaps
0,2023-05-25 12:45:00,2023-05-12 15:15:00,12 days 21:30:00,1237
1,2023-10-21 10:00:00,2023-10-20 18:30:00,0 days 15:30:00,61


Total Gaps: 1298

FILE: dta_YorkU_June2024(Credit MGCC).csv
LEN: 70223


...

Total Gaps: 39

FILE: dta_YorkU_June2024(Shaws Cr).csv
LEN: 70266


,Date,Date_Prev,Gap,Num_Gaps
0,2023-04-02 10:00:00,2023-04-02 09:15:00,0 days 00:45:00,2
1,2023-09-30 09:00:00,2023-09-30 07:45:00,0 days 01:15:00,4


Total Gaps: 6

FILE: dta_YorkU_June2024(West Credit 10th).csv
LEN: 70268


,Date,Date_Prev,Gap,Num_Gaps
0,2022-08-20 20:45:00,2022-08-20 19:30:00,0 days 01:15:00,4


Total Gaps: 4



## Hydrometric Discharge Missing Values

In [59]:
gap_choice = 15
folder_path = 'Ontario_Data/Credit_River_Conservation/Hydrometric'
for file in os.listdir(folder_path):
    print(f'FILE: {file}')
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=',', low_memory=False)
    try:
        df = df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis='columns')
    except:
        pass
    try:
        df['Date.1']
    except:
        continue
    df_discharge = df.iloc[:, 3:].copy()  # Use .copy() to avoid SettingWithCopyWarning
    df_discharge['Datetime'] = pd.to_datetime(df_discharge['Date.1'] + ' ' + df_discharge['Time.1'], format='%m/%d/%Y %I:%M:%S %p')
    df_discharge = df_discharge[df_discharge.notna().any(axis=1)]
    df_discharge = df_discharge[(df_discharge['Datetime'].dt.month >= 4) & (df_discharge['Datetime'].dt.month <= 11)]
    df_gap = pd.DataFrame(columns=['Date', 'Date_Prev', 'Gap', 'Num_Gaps'])
    print(f'LEN: {len(df_discharge.Datetime.unique())}')
    rows = []
    date_prev = 0
    for i, date in enumerate(df_discharge['Datetime'].unique()):
        if(i > 0):
            gap = date - date_prev
            if((gap.seconds/60 > gap_choice or gap.days > 0) and gap.days < 121):
                rows.append([date, date_prev, gap, int((gap.seconds/60 + gap.days*24*60)/15)-1])
        date_prev = date
    for row in rows:
        df_gap.loc[len(df_gap)] = row
    display(df_gap)
    print(f'Total Gaps: {df_gap.Num_Gaps.sum()}\n')

FILE: dta_YorkU_June2024(Credit Grange).csv
LEN: 70258


,Date,Date_Prev,Gap,Num_Gaps
0,2023-10-27 17:45:00,2023-10-27 14:00:00,0 days 03:45:00,14


Total Gaps: 14

FILE: dta_YorkU_June2024(Credit King).csv
LEN: 82393


,Date,Date_Prev,Gap,Num_Gaps
0,2021-10-25 09:20:00,2021-10-25 08:55:00,0 days 00:25:00,0
1,2023-05-18 07:30:00,2023-05-18 04:00:00,0 days 03:30:00,13
2,2023-05-25 06:45:00,2023-05-25 05:15:00,0 days 01:30:00,5


Total Gaps: 18

FILE: dta_YorkU_June2024(Credit MGCC).csv
LEN: 70225


...

Total Gaps: 1170

FILE: dta_YorkU_June2024(Serson Cr).csv
FILE: dta_YorkU_June2024(Shaw's Cr.csv
LEN: 70268


,Date,Date_Prev,Gap,Num_Gaps
0,2023-04-02 09:45:00,2023-04-02 09:15:00,0 days 00:30:00,1
1,2023-09-23 23:45:00,2023-09-23 23:15:00,0 days 00:30:00,1
2,2023-09-30 08:45:00,2023-09-30 08:00:00,0 days 00:45:00,2


Total Gaps: 4

FILE: dta_YorkU_June2024(West Credit 10th).csv
LEN: 70272


,Date,Date_Prev,Gap,Num_Gaps


Total Gaps: 0

